https://github.com/openai/openai-agents-python/blob/main/examples/basic/agent_lifecycle_example.py 

two agents
each has its own tools 

1. starter agents: will process the initial user request, will use tools as needed and will handoff to other agents if required
2. multiply agent: knows how to perform arithmetic operations (can we configure a handoff)

example also keeps tracks of agent life cycle events 


In [1]:
from dotenv import load_dotenv
from agents import (

    Agent,
    Runner,
    RunConfig,
    trace,
    ModelSettings
)

from pydantic import BaseModel, Field
load_dotenv()

True

In [2]:
from agents import Agent, RawResponsesStreamEvent, Runner, TResponseInputItem, trace
from helpers.trace_util import get_trace_url
from agents.extensions.visualization import draw_graph

In [3]:
from helpers.model_client import (
    get_openai_client,
    get_github_model_provider
)

model_name = "gpt-4o-mini"

GITHUB_MODEL_PROVIDER = get_github_model_provider(
    client = get_openai_client(),
    model = model_name
)

In [47]:
from agents.extensions.handoff_prompt import RECOMMENDED_PROMPT_PREFIX
from agents import handoff, RunContextWrapper


class Joke(BaseModel):
    joke: str = Field(description="The joke text")
    punchline: str = Field(description="The punchline of the joke")
    language: str = Field(description="The language of the joke")


class Translation(BaseModel):
    content: str = Field(description="Content")
    language: str = Field(description="The language of the translation")

async def on_handoff1(ctx: RunContextWrapper[None], input_data: Translation):
    print(f"agent {ctx} called with: {input_data.content}")

dad_jokes_agent = Agent(
    name="dad_jokes_generator_agent",
    instructions=(
        # f"{RECOMMENDED_PROMPT_PREFIX} \n"
        "You are an agent that specialises in generating jokes, dad jokes specifically. "
        "You only respond with dad jokes."
        "You do not answer any other questions. "
        "always deliver a punchline. "
        " ## IMPORTANT ##"
        "Always respond in English, even if the user asks in a different language. "
        "You don't end the conversation, you must handoff to another agent."
    ),
    handoff_description=(
        "An agent capable of generating jokes in English"
    ),
    model=model_name,
    output_type=Joke,
)


spanish_agent = Agent(
    name="spanish_agent",
    instructions= """
        You translate the input message to Spanish.
        You don't generate new content.
        ## IMPORTANT ##
        You don't end the conversation, you must handoff to another agent.
        only handoff to a single agent at a time

        """,
    handoff_description="""
    You translate content to spanish.
    """,
    model=model_name,
    output_type=Translation,

)



french_agent = Agent(
    name="french_agent",
    instructions= """
        You translate the input message to French.
        You don't generate new content.
        ## IMPORTANT ##
        You don't end the conversation, you must handoff to another agent.
        only handoff to a single agent at a time


        """,
    handoff_description="you translate content to french",
    model=model_name,
    output_type=Translation,
)





# Executing a Manual Handoff

In this example, we will execute a manual handoff between agents in a sequential workflow. 

All of the agents produce structured outputs. 

The output of one agent is used as the input to the next agent, after appending to it an additional handoff instruction message.

Notice in the trace afterwards how the full message histtory is preserved, including the handoff instruction message.

In [48]:

def handoff_instructions_msg(lang: str):
    return {"role": "user", "content": f"Translate the following text to {lang}"}

In [27]:
input_list = [
    {"role": "user", "content": "Tell me a dad joke"},
]


with trace("dad_jokes_agent_manual_handoff", group_id="dad_jokes") as tr:

    print(f"Trace URL: {get_trace_url(tr)}")

    run_output = await Runner.run(
        starting_agent= dad_jokes_agent,
        input=input_list,
        run_config = RunConfig(
            model_provider=GITHUB_MODEL_PROVIDER,
            model = model_name,
        ),
        )


    print(run_output)

    # joke_output : Joke = run_output.final_output
    # print(f"Joke: {joke_output.joke}")
    print(f"First joke as Joke: {run_output.final_output_as(Joke)}")


    next_input = run_output.to_input_list()
    next_input.append(handoff_instructions_msg("Spanish"))

    spanish_agent.output_type = Joke
    spanish_output = await Runner.run(
        starting_agent=spanish_agent,
        input=next_input,
        run_config = RunConfig(
            model_provider=GITHUB_MODEL_PROVIDER,
            model = model_name,
        ),
        )

    print(f"Spanish translation as Joke: {spanish_output.final_output_as(Joke)}")


    french_agent.output_type = Joke
    french_agent_input = spanish_output.to_input_list()
    french_agent_input.append(handoff_instructions_msg("French"))
    french_output = await Runner.run(
        starting_agent=french_agent,
        input=french_agent_input,
        run_config = RunConfig(
            model_provider=GITHUB_MODEL_PROVIDER,
            model = model_name,
        ),
        )


    print(f"French translation as Joke: {french_output.final_output_as(Joke)}")

    french_output.to_input_list()


print(french_output.final_output_as(Joke))

Trace URL: https://platform.openai.com/traces/trace?trace_id=trace_995a53902c994ced941e950226be31ff
RunResult:
- Last agent: Agent(name="dad_jokes_generator_agent", ...)
- Final output (Joke):
    {
      "joke": "Why did the scarecrow win an award?",
      "punchline": "Because he was outstanding in his field!",
      "language": "English"
    }
- 1 new item(s)
- 1 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)
First joke as Joke: joke='Why did the scarecrow win an award?' punchline='Because he was outstanding in his field!' language='English'
Spanish translation as Joke: joke='¿Por qué ganó un premio el espantapájaros?' punchline='¡Porque era excepcional en su campo!' language='Spanish'
French translation as Joke: joke="Pourquoi l'épouvantail a-t-il gagné un prix ?" punchline="Parce qu'il était exceptionnel dans son champ !" language='French'
joke="Pourquoi l'épouvantail a-t-il gagné un prix ?" punchline="Parce qu'il ét

In [28]:
print(french_output.raw_responses)

next_input = french_output.to_input_list()
display(next_input)

[ModelResponse(output=[ResponseOutputMessage(id='__fake_id__', content=[ResponseOutputText(annotations=[], text='{"joke":"Pourquoi l\'épouvantail a-t-il gagné un prix ?","punchline":"Parce qu\'il était exceptionnel dans son champ !","language":"French"}', type='output_text')], role='assistant', status='completed', type='message')], usage=Usage(requests=1, input_tokens=241, output_tokens=37, total_tokens=278), referenceable_id=None)]


[{'role': 'user', 'content': 'Tell me a dad joke'},
 {'id': '__fake_id__',
  'content': [{'annotations': [],
    'text': '{"joke":"Why did the scarecrow win an award?","punchline":"Because he was outstanding in his field!","language":"English"}',
    'type': 'output_text'}],
  'role': 'assistant',
  'status': 'completed',
  'type': 'message'},
 {'role': 'user', 'content': 'Translate the following text to Spanish'},
 {'id': '__fake_id__',
  'content': [{'annotations': [],
    'text': '{"joke":"¿Por qué ganó un premio el espantapájaros?","punchline":"¡Porque era excepcional en su campo!","language":"Spanish"}',
    'type': 'output_text'}],
  'role': 'assistant',
  'status': 'completed',
  'type': 'message'},
 {'role': 'user', 'content': 'Translate the following text to French'},
 {'id': '__fake_id__',
  'content': [{'annotations': [],
    'text': '{"joke":"Pourquoi l\'épouvantail a-t-il gagné un prix ?","punchline":"Parce qu\'il était exceptionnel dans son champ !","language":"French"}',

# Implement Handoffs

In this example, we will implement the handoffs using the SDK handoff functionality.

In [49]:

user_response_agent = Agent(
    name="user_response_agent",
    instructions="""
        You are an agent that generates a response to the user's message.
        your response should include summary of steps taken to produce the final output
        in bullet points.
        present this execution summary as a list of steps taken to produce the final output."
        then in the response back to the user, include the final output from each step,
        the user query, and the very last final output"
    """,
    handoff_description="""
    An agent capable of generating final user responses
    Final user response is a summary of the steps taken to produce the final output.
    """,
    model=model_name,
    output_type=str,
)


orchestrator_agent = Agent(
    name="orchestrator_agent",
    model=model_name,
    tool_use_behavior="run_llm_again",
    instructions="""
        You are an orchestrator agent.
        You breakdown the task to multiple steps and handoff to appropriate agents.
        You are capable of generating jokes, translating messages to Spanish and French.
        If asked for multiple translations, handoff to multiple language agents.
        You only speak English. Do not translate the user's message yourself.
        Use translator only when you have content to be translated, otherwise, handoff to the next agent.
        ## IMPORTANT ##
        Express your thoughts before executing the next step
    """,
    handoff_description=(
        "An agent that orchestrates tasks."
        ),
    # handoffs=[dad_jokes_agent,user_response_agent]
)



# # orchestrator_agent.handoffs = [dad_jokes_agent,user_response_agent]
# dad_jokes_agent.handoffs = [spanish_agent, french_agent, user_response_agent]
# spanish_agent.handoffs = [user_response_agent]
# french_agent.handoffs = [user_response_agent]

dad_jokes_agent.handoffs = [
    handoff(orchestrator_agent),
    # spanish_agent,
    # french_agent,
    ]

spanish_agent.handoffs = [handoff(orchestrator_agent)]

orchestrator_agent.handoffs = [
    dad_jokes_agent,
    spanish_agent,
    user_response_agent,
    ]


# # Handoff callback that processes the escalation data
async def process_handoff(ctx: RunContextWrapper, input_data: Joke):
   print(f"[handoff] Joke: {input_data.joke}")
   print(f"[handoff] Punchline: {input_data.punchline}")
   print(f"[handoff] Language: {input_data.language}")


# dad_jokes_agent.handoffs = [
#    handoff(
#          agent=spanish_agent,
#          input_type=Joke,
#          on_handoff=process_handoff,
#    ),
#    handoff(
#          agent=french_agent,
#          input_type=Joke,
#          on_handoff=process_handoff,
#    ),
#     handoff(
#             agent=user_response_agent,
#             input_type=Joke,
#             on_handoff=process_handoff,
#     ),
# ]


try:
    gv = draw_graph(
        orchestrator_agent,
        filename="./viz/07_agent_handoffs_orchestrator_agent.gv",

    )

except Exception as e:
    print(f"Error: {e}")



Error: failed to execute WindowsPath('dot'), make sure the Graphviz executables are on your systems' PATH


In [50]:
orchestrator_agent.handoffs
len(dad_jokes_agent.handoffs)
dad_jokes_agent.handoffs[0]


Handoff(tool_name='transfer_to_orchestrator_agent', tool_description='Handoff to the orchestrator_agent agent to handle the request. An agent that orchestrates tasks.', input_json_schema={'additionalProperties': False, 'type': 'object', 'properties': {}, 'required': []}, on_invoke_handoff=<function handoff.<locals>._invoke_handoff at 0x000002644B9F2320>, agent_name='orchestrator_agent', input_filter=None, strict_json_schema=True)

# Implementing Agent Handoffs with the Agents SDK

![handoff flow](./viz/07_agent_handoffs_orchestrator_agent.svg)

This visualization shows the agent flow we've created, where the orchestrator agent directs requests to the appropriate specialized agents. The dad jokes generator produces jokes, which can then be handed off to language agents (Spanish or French) for translation, and finally to the user response agent to format the final response.

In [52]:
from agents import Agent, RawResponsesStreamEvent, Runner, TResponseInputItem, trace

msg = input ("Enter your message: ") or "Hello, tell me a joke about chickens in spanish"

inputs: list[TResponseInputItem] = [{"content": msg, "role": "user"}]

result = None
try:
    with trace(workflow_name="handoffs", group_id="mdsi") as tr:
        print(f"Trace URL: {get_trace_url(tr)}")
        result = await Runner.run(
            starting_agent=orchestrator_agent,
            input=inputs,
            run_config=RunConfig(
                model=model_name,
                model_provider=GITHUB_MODEL_PROVIDER,
                model_settings=ModelSettings(
                    temperature=0.5,
                    max_tokens=5000,
                ),
                ),

            max_turns=10
        )

        print(result.final_output)
        # print(result.last_agent)
except Exception as e:
    print(f"Error: {e}")







Trace URL: https://platform.openai.com/traces/trace?trace_id=trace_b3ca8cd78c2e4f11a1fa14b0579b026a
Error: Max turns (10) exceeded


In [11]:
print(result)

print(get_trace_url(tr))

RunResult:
- Last agent: Agent(name="Spanish agent", ...)
- Final output (Joke):
    {
      "joke": "¿Por qué los pájaros no usan Facebook?",
      "punchline": "Porque ya tienen Twitter.",
      "language": "español"
    }
- 14 new item(s)
- 6 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)
https://platform.openai.com/traces/trace?trace_id=trace_5e57bf4fc1424924a24b76ede78925b4


In [ ]:
result.raw_responses

In [ ]:
from helpers.trace_util import display_agent_execution_steps


display_agent_execution_steps(result)

In [ ]:
tracing_url = f"https://platform.openai.com/traces/trace?trace_id={tr.trace_id}"

print(f"Tracing URL: {tracing_url}")

## Visualise the Agent

In [ ]:
from agents.extensions.visualization import draw_graph

try:
    draw_graph(orchestrator_agent, filename="viz/07_agent_handoffs.gv")
except Exception as e:
    print(f"An error occurred: {e}")

# another explains

In [ ]:

french_agent = Agent(
    name="french_agent",
    instructions="You only speak French",
)

spanish_agent = Agent(
    name="spanish_agent",
    instructions="You only speak Spanish",
)

english_agent = Agent(
    name="english_agent",
    instructions="You only speak English",
)

triage_agent = Agent(
    name="triage_agent",
    instructions="Handoff to the appropriate agent based on the language of the request.",
    handoffs=[french_agent, spanish_agent, english_agent],
)


In [ ]:
from openai.types.responses import ResponseContentPartDoneEvent, ResponseTextDeltaEvent

from agents import Agent, RawResponsesStreamEvent, Runner, TResponseInputItem, trace
import uuid

conversation_id = str(uuid.uuid4().hex[:16])

msg = input("Hi! We speak French, Spanish and English. How can I help? ")
agent = triage_agent
inputs: list[TResponseInputItem] = []
inputs.append({"content": msg, "role": "user"})

while True:
    # Each conversation turn is a single trace. Normally, each input from the user would be an
    # API request to your app, and you can wrap the request in a trace()




    with trace("Routing example", group_id=conversation_id):
        result = Runner.run_streamed(
            agent,
            input=inputs,
            run_config=RunConfig(model=model_name, model_provider=GITHUB_MODEL_PROVIDER),
        )
        async for event in result.stream_events():
            if not isinstance(event, RawResponsesStreamEvent):
                continue
            data = event.data
            if isinstance(data, ResponseTextDeltaEvent):
                print(data.delta, end="", flush=True)
            elif isinstance(data, ResponseContentPartDoneEvent):
                print("\n")

    inputs = result.to_input_list()
    print("\n")


    agent = result.current_agent
    user_msg = input("Enter a message: ")
    if user_msg in ("exit", "quit", ""):
        break
    inputs.append({"content": user_msg, "role": "user"})

In [ ]:


try:
    draw_graph(orchestrator_agent, filename="viz/07_agent_handoffs.gv")
except Exception as e:
    print(f"An error occurred: {e}")